## Problem
You are given an integer array matches where matches[i] = [winner<sub>i</sub>, loser<sub>i</sub>] indicates that the player winner<sub>i</sub> defeated player loser<sub>i</sub> in a match.

Return a list answer of size 2 where:
- answer[0] is a list of all players that have not lost any matches.
- answer[1] is a list of all players that have lost exactly one match.

The values in the two lists should be returned in **increasing** order.

Note:
- You should only consider the players that have played at least one match.
- The testcases will be generated such that no two matches will have the same outcome.

**Example 1:**
Input: matches = [[1,3],[2,3],[3,6],[5,6],[5,7],[4,5],[4,8],[4,9],[10,4],[10,9]]  
Output: [[1,2,10],[4,5,7,8]]  
Explanation:  
Players 1, 2, and 10 have not lost any matches.  
Players 4, 5, 7, and 8 each have lost one match.  
Players 3, 6, and 9 each have lost two matches.  
Thus, answer[0] = [1,2,10] and answer[1] = [4,5,7,8].  

**Example 2:**
Input: matches = [[2,3],[1,3],[5,4],[6,4]]  
Output: [[1,2,5,6],[]]  
Explanation:  
Players 1, 2, 5, and 6 have not lost any matches.  
Players 3 and 4 each have lost two matches.  
Thus, answer[0] = [1,2,5,6] and answer[1] = [].  

**Constraints:**
- 1 <= matches.length <= 10<sup>5</sup>
- matches[i].length == 2
- 1 <= winner<sub>i</sub>, loser<sub>i</sub> <= 10<sup>5</sup>
- winner<sub>i</sub> != loser<sub>i</sub>
- All matches[i] are unique.

## Approach 1: Hash Set
Create 3 hash set to store players with different losses. (zero loss, one loss, more loss)

For each iteration, or each match, 2 players (winner and loser) may need to move to other sets and we update the sets. That is remove the current player from the current set (zero loss or one loss) and move it to the next container (one loss or more loss)

### Algorithm
1. Initialize three empty hash sets, zero_loss, one_loss and more_losses.
2. Iterate over matches, for each match [winner, loser], update the sets they are in according to the following rule:
  - For Winner:
    - If winner is not in more_losses or one_loss, it means he should be in zero_loss, add it to zero_loss.
    - Otherwise, the number of losses for winner doesn't change, do not change the player.
  - For Loser:
    - If loser is in zero_loss, remove it from zero_loss and add it to one_loss since this player has one more loss now.
    - If loser is in one_loss, remove it from one_loss and add it to more_losses since this player has one more loss now.
    - If loser is in more_losses, keep this player in more_losses.
    - Otherwise, if loser can't be found in those three containers, then this match is loser's first match, we add this player to one_loss.
3. After the iteration ends, get the players from zero_loss and one_loss and sort them.

### Complexity
Time Complexity: O(n logn), iteration over matches takes O(n) time, and the dominant sorting takes O(n logn) time.  
Space Complexity: O(n), use 3 hash map to store all players


In [ ]:
class Solution {
public:
    vector<vector<int>> findWinners(vector<vector<int>>& matches) {
        unordered_set<int> zeroLoss, oneLoss, moreLoss;

        for (auto& match : matches) {
            int winner = match[0], loser = match[1];
            // Add winner.
            if ((oneLoss.find(winner) == oneLoss.end()) &&
                (moreLoss.find(winner) == moreLoss.end())) {
                zeroLoss.insert(winner);
            }
            // Add or move loser.
            if (zeroLoss.find(loser) != zeroLoss.end()) {
                zeroLoss.erase(loser);
                oneLoss.insert(loser);
            } else if (oneLoss.find(loser) != oneLoss.end()) {
                oneLoss.erase(loser);
                moreLoss.insert(loser);
            } else if (moreLoss.find(loser) != moreLoss.end()) {
                continue;
            } else {
                oneLoss.insert(loser);
            }
        }

        vector<vector<int>> answer(2, vector<int>());
        answer[0].assign(zeroLoss.begin(), zeroLoss.end());
        answer[1].assign(oneLoss.begin(), oneLoss.end());
        sort(answer[0].begin(), answer[0].end());
        sort(answer[1].begin(), answer[1].end());

        return answer;
    }
};

In [ ]:
from typing import List

class Solution:
    def findWinners(self, matches: List[List[int]]) -> List[List[int]]:
        zero_loss = set()
        one_loss = set()
        more_losses = set()
        
        for winner, loser in matches:
            # Add winner
            if (winner not in one_loss) and (winner not in more_losses):
                zero_loss.add(winner)
            # Add or move loser.
            if loser in zero_loss:
                zero_loss.remove(loser)
                one_loss.add(loser)
            elif loser in one_loss:
                one_loss.remove(loser)
                more_losses.add(loser)
            elif loser in more_losses:
                continue
            else:
                one_loss.add(loser)          
            
        return [sorted(list(zero_loss)), sorted(list(one_loss))]

In [ ]:
class Solution {
    public List<List<Integer>> findWinners(int[][] matches) {
        Set<Integer> zeroLoss = new HashSet<>(), oneLoss = new HashSet<>(),
                moreLosses = new HashSet<>();

        for (int[] match : matches) {
            int winner = match[0], loser = match[1];
            // Add winner.
            if (!oneLoss.contains(winner) && !moreLosses.contains(winner)) {
                zeroLoss.add(winner);
            }
            // Add or move loser.
            if (zeroLoss.contains(loser)) {
                zeroLoss.remove(loser);
                oneLoss.add(loser);
            } else if (oneLoss.contains(loser)) {
                oneLoss.remove(loser);
                moreLosses.add(loser);
            } else if (moreLosses.contains(loser)) {
                continue;
            } else {
                oneLoss.add(loser);
            }
        }

        List<List<Integer>> answer =
            Arrays.asList(new ArrayList<>(), new ArrayList<>());
        answer.get(0).addAll(zeroLoss);
        answer.get(1).addAll(oneLoss);
        Collections.sort(answer.get(0));
        Collections.sort(answer.get(1));

        return answer;
    }
}

## Approach 2: Hash Set + Hash Map
The previous methods works but has extremely limited usablility. k could be possibly very large in other questions.

We can use a **hash map** losses_count to store players and the number of losses each has.  
Hence, we use a **hash set** seen to store all the players, and a hash map losses_count to store the number of losses each loser has.

### Algorithm
1. Initialize an empty hash set seen and an empty hash map losses_count.  
2. Iterate over matches and for each match [winner, loser], add both winner and loser to seen. Increment losses_count[loser] by 1.
3. After the iteration stops, we iterate through all the players in seen and collect players with 0 loss or 1 loss to two arrays respectively.
4. Sort these two arrays.

### Complexity
Time Complexity: O(n logn), iteration over match takes O(n), Store and store takes O(n logn)  
Space Complexity: O(n), use hash map and hash set to store all the players

In [ ]:
class Solution {
public:
    vector<vector<int>> findWinners(vector<vector<int>>& matches) {
        unordered_set<int> seen;
        unordered_map<int, int> lossesCount;

        for (auto& match : matches) {
            int winner = match[0], loser = match[1];
            seen.insert(winner);
            seen.insert(loser);
            lossesCount[loser]++;
        }

        // Add players with 0 or 1 loss to the corresponding list.
        vector<vector<int>> answer(2, vector<int>());
        for (auto player : seen) {
            if (lossesCount.find(player) == lossesCount.end()) {
                answer[0].push_back(player);
            } else if (lossesCount[player] == 1) {
                answer[1].push_back(player);
            }
        }

        sort(answer[0].begin(), answer[0].end());
        sort(answer[1].begin(), answer[1].end());
        return answer;
    }
};

In [ ]:
class Solution: 
    def findWinners(self, matches : List[List[int]]) ->List[List[int]]: 
        seen = set()
        losses_count = {}
        
        for winner, loser in matches:
            seen.add(winner)
            seen.add(loser)
            losses_count[loser] = losses_count.get(loser, 0) + 1
        
        #Add players with 0 or 1 loss to the corresponding list.
        zero_lose, one_lose = [], []
        for player in seen:
            count = losses_count.get(player, 0)
            if count == 0:
                zero_lose.append(player)
            elif count == 1:
                one_lose.append(player)
        
        return [sorted(zero_lose), sorted(one_lose)]

In [ ]:
class Solution {
    public List<List<Integer>> findWinners(int[][] matches) {
        Set<Integer> seen = new HashSet<>();
        Map<Integer, Integer> lossesCount = new HashMap<>();

        for (int[] match : matches) {
            int winner = match[0], loser = match[1];
            seen.add(winner);
            seen.add(loser);
            lossesCount.put(loser, lossesCount.getOrDefault(loser, 0) + 1);
        }

        // Add players with 0 or 1 loss to the corresponding list.
        List<List<Integer>> answer =
            Arrays.asList(new ArrayList<>(), new ArrayList<>());
        for (int player : seen) {
            if (!lossesCount.containsKey(player)) {
                answer.get(0).add(player);
            } else if (lossesCount.get(player) == 1) {
                answer.get(1).add(player);
            }
        }

        Collections.sort(answer.get(0));
        Collections.sort(answer.get(1));

        return answer;
    }
}

## Approach 3: Hash Map
An improvement on approach 2: 2e can also store the players with 0 loss in the same hash map, so we no longer need the hash set seen to store all the players

For each [winner, loser]:
- We increment loser's number of losses by 1.
- if winner has 1 more loss already, += 1, otherwise, set its value to 0, show that he haven't lost a match yet.

### Algorithm
1. Initialize a map called lossesCount to track the number of losses for each player.
2. Iterate over each match:
- Extract the winner and loser from the match.
- Update lossesCount:
  - Set the number of losses for winner to 0 if not already present.
  - Increment the number of losses for loser by 1.
3. Initialize a list of list of integer called answer with two empty lists: The first list will store players with 0 losses. The second list will store players with exactly 1 loss.
4. Iterate over the lossesCount map:
- If a player's loss count is 0, add the player to the first list in answer.
- If a player's loss count is 1, add the player to the second list in answer.
5. Sort both lists in answer to ensure the output is in ascending order.
6. Return the answer list containing the two sorted lists of players.

### Compelxity
Time Complexity: O(n logn) -> For each match, need to update the value for both players, that's O(n). Sorting takes O(n logn) time.  
Space Complexity: O(n) -> hashmap

In [ ]:
class Solution {
public:
    vector<vector<int>> findWinners(vector<vector<int>>& matches) {
        unordered_map<int, int> lossesCount;
        for (auto& match : matches) {
            int winner = match[0], loser = match[1];
            if (lossesCount.find(winner) == lossesCount.end()) {
                lossesCount[winner] = 0;
            }
            lossesCount[loser]++;
        }

        vector<vector<int>> answer(2, vector<int>());
        for (auto [player, count] : lossesCount) {
            if (count == 0) {
                answer[0].push_back(player);
            } else if (count == 1) {
                answer[1].push_back(player);
            }
        }

        sort(answer[0].begin(), answer[0].end());
        sort(answer[1].begin(), answer[1].end());
        return answer;
    }
};

In [ ]:
class Solution: 
    def findWinners(self, matches: List[List[int]]) ->List[List[int]]: 
        losses_count = {}
        
        for winner, loser in matches:
            losses_count[winner] = losses_count.get(winner, 0)
            losses_count[loser] = losses_count.get(loser, 0) + 1
        
        zero_lose, one_lose = [], []
        for player, count in losses_count.items():
            if count == 0:
                zero_lose.append(player)
            if count == 1:
                one_lose.append(player)
        
        return [sorted(zero_lose), sorted(one_lose)]

In [ ]:
class Solution {
    public List<List<Integer>> findWinners(int[][] matches) {
        Map<Integer, Integer> lossesCount = new HashMap<>();
        for (int[] match : matches) {
            int winner = match[0], loser = match[1];
            lossesCount.put(winner, lossesCount.getOrDefault(winner, 0));
            lossesCount.put(loser, lossesCount.getOrDefault(loser, 0) + 1);
        }

        List<List<Integer>> answer =
            Arrays.asList(new ArrayList<>(), new ArrayList<>());
        for (Integer player : lossesCount.keySet())
            if (lossesCount.get(player) == 0) {
                answer.get(0).add(player);
            } else if (lossesCount.get(player) == 1) {
                answer.get(1).add(player);
            }

        Collections.sort(answer.get(0));
        Collections.sort(answer.get(1));

        return answer;
    }
}

## Approach 4: Counting with array
mapping each of the players to an (unique) index within a specific range.
- losses_count[i] = -1, player i has not played.
- losses_count[i] = 0, player i has played at least one game and has 0 loss.
- losses_count[i] = 1, player i has exact 1 loss.
- losses_count[i] > 1, player i has more than 1 loss.

### Algorithm
1. Use an array losses_count to store the number of losses for each player. Initially, losses_count[i] = -1 for every index i.
2. For each match [winner, loser]:
- If losses_count[loser] = -1, set it to 1, otherwise increment it by 1.
- If losses_count[winner] = -1, set it to 0.
3. Iterate over losses_count and use two arrays to store these 2 kinds of players, for each index i:
- If losses_count[i] = 0, add this player to the first array.
- If losses_count[i] = 1, add this player to the second array.

### Complexity
Time Complexity: O(n + k) -> For each match, we need to update two values in the array losses_count which takes constant time. Thus the iteration requires O(n) time. We need to iterate over losses_count to collect two kinds of players, which takes O(k) time.  
Space Complexity: O(k) -> We need to create an array of size O(k) to cover all players.

In [ ]:
class Solution {
public:
    vector<vector<int>> findWinners(vector<vector<int>>& matches) {
        vector<int> lossesCount(100001, -1);

        for (auto& match : matches) {
            int winner = match[0], loser = match[1];
            if (lossesCount[winner] == -1) {
                lossesCount[winner] = 0;
            }
            if (lossesCount[loser] == -1) {
                lossesCount[loser] = 1;
            } else {
                lossesCount[loser]++;
            }
        }

        vector<vector<int>> answer(2, vector<int>());
        for (int i = 1; i < 100001; ++i) {
            if (lossesCount[i] == 0) {
                answer[0].push_back(i);
            } else if (lossesCount[i] == 1) {
                answer[1].push_back(i);
            }
        }

        return answer;
    }
};

In [ ]:
def findWinners(self, matches: List[List[int]]) -> List[List[int]]:
        losses_count = [-1] * 100001

        for winner, loser in matches:
            if losses_count[winner] == -1:
                losses_count[winner] = 0
            if losses_count[loser] == -1:
                losses_count[loser] = 1
            else:
                losses_count[loser] += 1
            
        answer = [[], []]
        for i in range(100001):
            if losses_count[i] == 0:
                answer[0].append(i)
            elif losses_count[i] == 1:
                answer[1].append(i)
                
        return answer

In [ ]:
class Solution {
    public List<List<Integer>> findWinners(int[][] matches) {
        int[] lossesCount = new int[100001];
        Arrays.fill(lossesCount, -1);

        for (int[] match : matches) {
            int winner = match[0], loser = match[1];
            if (lossesCount[winner] == -1) {
                lossesCount[winner] = 0;
            }
            if (lossesCount[loser] == -1) {
                lossesCount[loser] = 1;
            } else {
                lossesCount[loser]++;
            }
        }

        List<List<Integer>> answer =
            Arrays.asList(new ArrayList<>(), new ArrayList<>());
        for (int i = 1; i < 100001; ++i) {
            if (lossesCount[i] == 0) {
                answer.get(0).add(i);
            } else if (lossesCount[i] == 1) {
                answer.get(1).add(i);
            }
        }

        return answer;
    }
}